# **Clustering**

In this NB, we will apply what was learnt about tensor operations in the `01_matmul` notebook to classical ML algorithms - specifically unsupervised learning techniques like Clustering. 

The homework at the end of the course NB's is the key to this lesson. We should be able to extend tensor operations to other unsupervised algorithms such as dbscan, k-means clustering , fast nearest neighbours _while utilizing our GPU._ Implementing in APL is the bonus.

According to Jeremy, this is akin to knowing our times tables. So "time" to get cracking!

In [1]:
import math, matplotlib.pyplot as plt, operator, torch
from functools import partial

In [2]:
torch.manual_seed(42)
torch.set_printoptions(precision=3, linewidth=140, sci_mode=False)

## Create Synthetic Data